In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/parsed_data.csv")

In [3]:
df

,app,hostname,error_model,has_end,sdc,it,ker_time,cuda_framework_error,log_file,due_cause,was_fault_injected
0,darknet_v3,carolgtxtn,IAC,0,0,0,0.000000,0,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal memory access was encountered);,1
1,darknet_v3,carolgtxtn,IAC,0,0,0,0.000000,0,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal memory access was encountered);,1
2,darknet_v3,carolgtxtn,IAC,0,0,0,0.000000,0,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal memory access was encountered);,1
3,darknet_v3,carolgtxtn,IAC,0,0,0,0.000000,0,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal memory access was encountered);,1
4,darknet_v3,carolgtxtn,IAC,0,0,0,0.000000,0,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal memory access was encountered);,1
...,...,...,...,...,...,...,...,...,...,...,...
9995,darknet_v3,carolgtxtn,WV,1,0,0,5.658111,0,log_helper_dest/radiation-benchmarks/log/2022_...,NoDUE,1
9996,darknet_v3,carolgtxtn,WV,0,0,0,0.000000,0,log_helper_dest/radiation-benchmarks/log/2022_...,TIMEOUT,1
9997,darknet_v3,carolgtxtn,WV,0,0,0,0.000000,0,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal memory access was encountered);,1
9998,darknet_v3,carolgtxtn,WV,0,0,0,0.000000,0,log_helper_dest/radiation-benchmarks/log/2022_...,(an illegal memory access was encountered);,0


In [4]:
due_cause_names = {
 'NoDUE': "NoDUE",
 '::ERROR FAIL in kernel execution (an illegal memory access was encountered); ': "IllegalMemAccess",
 '::ERROR FAIL in kernel execution (misaligned address); ': "MisalignedAddress",
 'TIMEOUT': "Timeout",
 '::ERROR FAIL in kernel execution (operation not supported on global/shared address space); ': "OpNotSuppAddSpace",
    '(operation not supported on global/shared address space); ': "OpNotSuppAddSpace",
 '::ERROR FAIL in kernel execution (invalid program counter); ': "InvalidPC",
 '(an illegal memory access was encountered); ': "IllegalMemAccess",
 '(an illegal instruction was encountered); ': "IllegalInstruction",
 '(misaligned address); ':  "MisalignedAddress",
    '(invalid program counter); ':"InvalidPC"
}

In [5]:
df["due_cause"] = df["due_cause"].apply(lambda x: due_cause_names[x])

In [6]:
df["due"] = ((df["has_end"] == 0) & (df["due_cause"] != "NoDUE")).astype(int)

In [13]:
df[(df["due"] == 1) & (df["was_fault_injected"] == 0)]

,app,hostname,error_model,has_end,sdc,it,ker_time,cuda_framework_error,log_file,due_cause,was_fault_injected,due,count
1000,darknet_v3,carolgtxtn,IAL,0,0,0,0.0,0,log_helper_dest/radiation-benchmarks/log/2022_...,IllegalMemAccess,0,1,1
1002,darknet_v3,carolgtxtn,IAL,0,0,0,0.0,0,log_helper_dest/radiation-benchmarks/log/2022_...,IllegalMemAccess,0,1,1
1003,darknet_v3,carolgtxtn,IAL,0,0,0,0.0,0,log_helper_dest/radiation-benchmarks/log/2022_...,IllegalMemAccess,0,1,1
1004,darknet_v3,carolgtxtn,IAL,0,0,0,0.0,0,log_helper_dest/radiation-benchmarks/log/2022_...,IllegalMemAccess,0,1,1
1005,darknet_v3,carolgtxtn,IAL,0,0,0,0.0,0,log_helper_dest/radiation-benchmarks/log/2022_...,IllegalMemAccess,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9972,darknet_v3,carolgtxtn,WV,0,0,0,0.0,0,log_helper_dest/radiation-benchmarks/log/2022_...,IllegalMemAccess,0,1,1
9989,darknet_v3,carolgtxtn,WV,0,0,0,0.0,0,log_helper_dest/radiation-benchmarks/log/2022_...,IllegalMemAccess,0,1,1
9990,darknet_v3,carolgtxtn,WV,0,0,0,0.0,0,log_helper_dest/radiation-benchmarks/log/2022_...,IllegalMemAccess,0,1,1
9993,darknet_v3,carolgtxtn,WV,0,0,0,0.0,0,log_helper_dest/radiation-benchmarks/log/2022_...,IllegalMemAccess,0,1,1


In [7]:
df["count"] = 1

In [8]:
avf_df = df.groupby(["app", "error_model"]).sum()
due_df = df.groupby(["app", "error_model", "due_cause"]).sum()

In [9]:
avf_df = avf_df[["sdc", "due", "count", "was_fault_injected"]].copy()
due_df = due_df[["count", "was_fault_injected"]].copy()

In [10]:
avf_df["AVF SDC"] = avf_df["sdc"] / avf_df["count"]
avf_df["AVF DUE"] = avf_df["due"] / avf_df["count"]

In [11]:
with pd.ExcelWriter("parsed_avf.xlsx") as writer:
    avf_df.to_excel(writer, sheet_name="GeneralAVF")  
    due_df.to_excel(writer, sheet_name="DUESource")  

In [12]:
avf_df

sdc   due  count  was_fault_injected  AVF SDC  AVF DUE
app        error_model                                                        
darknet_v3 IAC           29   980   1000                1000    0.029    0.980
           IAL            0  1000   1000                 490    0.000    1.000
           IAT          622   378   1000                1000    0.622    0.378
           IAW          457   538   1000                1000    0.457    0.538
           ICOC           0   999   1000                1000    0.000    0.999
           IIO            0   999   1000                1000    0.000    0.999
           IMD          167   791   1000                 279    0.167    0.791
           IMS            0  1000   1000                1000    0.000    1.000
           IRA            0  1000   1000                  97    0.000    1.000
           WV           230   707   1000                 825    0.230    0.707